In [1]:
from ultralytics import YOLO
import os
import cv2
import numpy as np
import pandas as pd
from collections import defaultdict
from PIL import Image, ImageDraw
from tqdm import tqdm
import csv

In [2]:
model_path = r"C:\Users\sharm\Python Projects\ML Projects\Bengaluru Mobility Challenge\Bengaluru Mobility Challenge Solution\runs\detect\train\weights\best.pt"
yolo_model = YOLO(model_path)
print("Model loaded successfully!")

Model loaded successfully!


In [3]:
track_history = defaultdict(list)
vehicle_classes = defaultdict(int)

In [4]:
video_file = r"C:\Users\sharm\Python Projects\ML Projects\Bengaluru Mobility Challenge\Bengaluru Mobility Challenge Solution\Video File\Stn_HD_1_time_2024-05-29T07_30_02_002.mp4"
video_file

'C:\\Users\\sharm\\Python Projects\\ML Projects\\Bengaluru Mobility Challenge\\Bengaluru Mobility Challenge Solution\\Video File\\Stn_HD_1_time_2024-05-29T07_30_02_002.mp4'

In [5]:
cap = cv2.VideoCapture(video_file)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
duration = total_frames / fps
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"Total frames: {total_frames}")
print(f"Frame width : {frame_width}")
print(f"Frames height : {frame_height}")
print(f"Frames per second (FPS): {fps}")
print(f"Duration of video (in seconds): {duration:.2f}")

Total frames: 22503
Frame width : 1920
Frames height : 1080
Frames per second (FPS): 25.0
Duration of video (in seconds): 900.12


In [6]:
output_dir = r"C:\Users\sharm\Python Projects\ML Projects\Bengaluru Mobility Challenge\Bengaluru Mobility Challenge Solution\frames"
os.makedirs(output_dir, exist_ok=True)

In [7]:
frame_count = 0
progress_bar = tqdm(total=total_frames, desc='Processing Video', unit='frame', dynamic_ncols=True)

while cap.isOpened():
    success, frame = cap.read()
    #frame = cv2.resize(frame, (640, 384))
    if not success:
        break
        
    results = yolo_model.track(frame, tracker = 'bytetrack.yaml', persist=True, verbose = False)
    try:
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        classes = results[0].boxes.cls.int().cpu().tolist()
        annotated_frame = results[0].plot()
            
        for box, track_id, cls in zip(boxes, track_ids, classes):
            x, y, w, h = box
            track_history[track_id].append((cls, float(x), float(y)))
            vehicle_classes[track_id] = cls
                
            # Visualization (optional)
            points = np.array(track_history[track_id]).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=2)
            
        #frame_filename = f"frame_{frame_count:06d}.jpg"
        #frame_path = os.path.join(output_dir, frame_filename)
        #cv2.imwrite(frame_path, annotated_frame)
            
            
        frame_count += 1
        progress_bar.update(1)

        #cv2.namedWindow("Vehicle Tracking", cv2.WINDOW_NORMAL)
        #cv2.imshow("Vehicle Tracking", annotated_frame)
       
        
    except Exception as e:
        continue
    
    progress_bar.close()

cap.release()
cv2.destroyAllWindows()

Processing Video:   0%|          | 1/22503 [00:05<36:48:29,  5.89s/frame]


In [8]:
len(track_history)

1464

In [15]:
num = 1
for obj, frame in track_history.items():
    print(obj, frame, frame[0][0])
    num += 1
    if num > 5:
        break

1 [(2, 263.349365234375, 162.67593383789062), (2, 263.219482421875, 162.73721313476562), (2, 273.6424865722656, 157.7827606201172), (2, 281.81964111328125, 155.2445068359375), (2, 290.3223876953125, 154.3070068359375), (2, 299.1944885253906, 153.82322692871094), (2, 307.6299133300781, 152.47230529785156), (2, 322.173828125, 148.52447509765625), (2, 328.48388671875, 149.9545440673828), (2, 343.22998046875, 149.2075653076172), (2, 351.0227355957031, 149.1234130859375), (2, 364.15277099609375, 148.10784912109375), (2, 373.2123718261719, 146.74288940429688), (2, 388.86572265625, 145.60562133789062), (2, 397.525146484375, 142.24713134765625), (2, 408.311279296875, 141.1492919921875), (2, 415.84881591796875, 139.5624237060547), (2, 427.4529113769531, 140.33067321777344), (2, 437.0250549316406, 138.6474609375), (2, 444.29998779296875, 137.027587890625), (2, 453.2716064453125, 135.69857788085938), (2, 460.62158203125, 133.67552185058594), (2, 468.385009765625, 131.6337127685547), (2, 474.19451

In [17]:
track_df = pd.DataFrame([(obj_id, history[0][0], history) for obj_id, history in track_history.items()],
                            columns=["Object ID", "Class", "Track History"])
track_df

,Object ID,Class,Track History
0,1,2,"[(2, 263.349365234375, 162.67593383789062), (2..."
1,2,2,"[(2, 1755.1876220703125, 310.5132751464844), (..."
2,3,2,"[(2, 1102.65576171875, 101.754150390625), (2, ..."
3,4,2,"[(2, 608.111083984375, 249.60662841796875), (2..."
4,5,6,"[(6, 1440.5989990234375, 147.79611206054688), ..."
...,...,...,...
1459,4490,6,"[(6, 1216.593505859375, 218.48361206054688), (..."
1460,4499,6,"[(6, 762.932861328125, 39.01149368286133), (6,..."
1461,4503,2,"[(2, 1055.017578125, 19.89910125732422), (2, 1..."
1462,4504,6,"[(6, 1071.1376953125, 117.34809875488281), (6,..."


In [19]:
track_df.to_csv('track_history.csv', index=False)

In [21]:
df = pd.read_csv(r"C:\Users\sharm\Python Projects\ML Projects\Bengaluru Mobility Challenge\track_history.csv")
df

,Object ID,Class,Track History
0,1,2,"[(2, 263.349365234375, 162.67593383789062), (2..."
1,2,2,"[(2, 1755.1876220703125, 310.5132751464844), (..."
2,3,2,"[(2, 1102.65576171875, 101.754150390625), (2, ..."
3,4,2,"[(2, 608.111083984375, 249.60662841796875), (2..."
4,5,6,"[(6, 1440.5989990234375, 147.79611206054688), ..."
...,...,...,...
1459,4490,6,"[(6, 1216.593505859375, 218.48361206054688), (..."
1460,4499,6,"[(6, 762.932861328125, 39.01149368286133), (6,..."
1461,4503,2,"[(2, 1055.017578125, 19.89910125732422), (2, 1..."
1462,4504,6,"[(6, 1071.1376953125, 117.34809875488281), (6,..."
